In [133]:
import json


def workflow(stage=0):
    def workflow_decorator(fcn):
        def function_wrapper(self, *args, **kwargs):
            
            
            def descend_blockchain():
                for s in range(stage-1, -1, -1):
                    if s in self._blockchain:
                        return [x['hash_after'] for x in self._blockchain[s].values()]
                return [None]
            
            
            def check_blockchain():
                print('--- '+fcn.__name__)
                previous_hashes = set([None])
                for s, blockchain in sorted(self._blockchain.items()):
                    if s >= stage:
                        break
                        
                    hashes_before = set()
                    hashes_after = set()
                    fcnnames = []
                    for fcnname, bc in sorted(blockchain.items()):
                        hashes_before.update(bc['hashes_before'])
                        hashes_after.add(bc['hash_after'])
                        fcnnames.append(fcnname)
                        
                    print(s, stage, hashes_before, hashes_after, fcnnames)
                    if len(hashes_before.difference(previous_hashes)) == 0:
                        previous_hashes = hashes_after
                    else:
                        raise ValueError('Blockchain broken at stage %d, '
                                         'rerun "%s" first.' % (s, '" and/or "'.join(fcnnames)))
                return True
            
            
            def append_blockchain(fcnname, **kwargs):
                if stage not in self._blockchain.keys():
                    self._blockchain[stage] = {}
                self._blockchain[stage][fcnname] = kwargs
                    
                
            if check_blockchain():
                fcn(self, *args, **kwargs)
                append_blockchain(fcn.__name__,
                                  hashes_before=descend_blockchain(),
                                  hash_after=self.hash)
            
        return function_wrapper
    
    return workflow_decorator
    
    
class A:
    
    def __init__(self):
        self.hash = 0
        self._blockchain = {}
    
    @workflow(stage=1)
    def do_something(self, x):
        self.hash = x
        
    @workflow(stage=2)
    def do_something_similar(self, x):
        self.hash = 2.*x
        
    @workflow(stage=2)
    def do_something_else(self, x):
        self.hash = x**2.

    @workflow(stage=3)
    def do_another_thing(self, x):
        self.hash = x**3.
        
    @workflow(stage=4)
    def do_something_final(self, x):
        self.hash = x**4.
        
        
obj = A()

obj.do_something(1)
obj.do_something_similar(1.5)
obj.do_something_else(2)
obj.do_another_thing(3)
obj.do_something_final(4)

#obj.do_something(11)
obj.do_something_similar(15)
obj.do_something(111)
#obj.do_something_else(22)
obj.do_something_similar(155)
#obj.do_another_thing(33)
#obj.do_something_final(44)

--- do_something
--- do_something_similar
1 2 {None} {1} ['do_something']
--- do_something_else
1 2 {None} {1} ['do_something']
--- do_another_thing
1 3 {None} {1} ['do_something']
2 3 {1} {3.0, 4.0} ['do_something_else', 'do_something_similar']
--- do_something_final
1 4 {None} {1} ['do_something']
2 4 {1} {3.0, 4.0} ['do_something_else', 'do_something_similar']
3 4 {3.0, 4.0} {27.0} ['do_another_thing']
--- do_something_similar
1 2 {None} {1} ['do_something']
--- do_something
--- do_something_similar
1 2 {None} {111} ['do_something']


In [118]:
print(json.dumps(obj._blockchain, indent=4))

{
    "1": {
        "do_something": {
            "hashes_before": [
                null
            ],
            "hash_after": 111
        }
    },
    "2": {
        "do_something_similar": {
            "hashes_before": [
                1
            ],
            "hash_after": 30.0
        },
        "do_something_else": {
            "hashes_before": [
                111
            ],
            "hash_after": 484.0
        }
    },
    "3": {
        "do_another_thing": {
            "hashes_before": [
                30.0,
                484.0
            ],
            "hash_after": 35937.0
        }
    },
    "4": {
        "do_something_final": {
            "hashes_before": [
                35937.0
            ],
            "hash_after": 3748096.0
        }
    }
}


In [128]:
a = set([1,2,3])
b = set([2,3,4])
a.intersection(b)
a.difference(b)

{1}

In [ ]:
slice()